In [6]:
# import all packages
import os, platform
import subprocess
import shutil
import nltk
from nltk import *

This module already contains a /pdf for all IMF Article IV files for analysis. 
I proceed by reading /pdf to new txt folder named /tmp 

In [7]:
# PROCESS PDF TO TXT

# pdf files dir
input = os.getcwd() + '/pdf/'
# final dir after converted
output = os.getcwd() + '/tmp/'
# check/create if output folder exist
if not os.path.exists(output):
   os.makedirs(output)

# process pdf to txt file save in /tmp
# check platform system Windows/Macs
xpdfPath = ''
if platform.system() == 'Windows':
     xpdfPath = os.getcwd() + '\\xpdfbin-win-3.04\\bin64\\'
else:
     xpdfPath = os.getcwd() + '/xpdfbin-mac-3.04/bin64/'

#print("xpdf: " + xpdfPath)
# store .pdf files from input
doc_list = [pdf for pdf in os.listdir(input) if pdf.endswith('.pdf')]
# iterate and process doc_list to txt file
print(doc_list)
for doc in doc_list:
    #print(input+doc)
    #print(output)
    subprocess.call(xpdfPath+'pdftotext ' + input + doc + ' ' + output + doc.replace('.pdf', '.txt'),
                        shell=True)


['-doc-cr1709.pdf', '-doc-cr1711.pdf', '-doc-cr1715.pdf', '-doc-cr1719.pdf', '-doc-cr1722.pdf', '-doc-cr1723.pdf', '-doc-cr1726.pdf', '-doc-cr1728.pdf', '-doc-cr1730.pdf', '-doc-cr1732.pdf', '-doc-cr1736.pdf', '-doc-cr1737.pdf', '-doc-cr1739.pdf', '-doc-cr1740.pdf', '-doc-cr1742.pdf', '-doc-cr1750.pdf', '-doc-cr1753.pdf', '_cr15178.pdf']


Once I have a /tmp with all txt files, I start strip out Countries' names from within 
the document to save them as file names to an actual /txt folder.

In [8]:
# SET FILE NAME TO COUNTRY--YEAR

# clear/retrieve txt file
txt_list = [txt for txt in os.listdir(output) if txt.endswith('.txt')]
#print(txt_list)
newOutput = os.getcwd() + '/txt/'
# iterate and process txt_list to new txt folder
for f in txt_list:
    filepath=output+f
    with open(filepath,encoding="ascii",errors='ignore') as fp:
        lp = fp.readlines()
        fileName = ''
        docDate = ''
        for i, line in enumerate(lp):
            print(line)
           # retrieve doc date from 1st line
            if i == 0:
           # print(line.replace('\n', ''))
                docDate = line.replace('\n', '')
           # retrieve doc title from range(2,4)
            if i in range(2, 4) and 'IMF' not in line:
              # make sure line is not an empty line
                if line.strip():
                    fileName = line.replace('\n', '') + '--' + docDate
            if i > 4:
                break
                
        if not os.path.exists(newOutput):
            os.makedirs(newOutput)

        shutil.copy(output + f, newOutput + fileName + '.txt')

January 2017



IMF Country Report No. 17/9

REPUBLIC OF ESTONIA

2016 ARTICLE IV CONSULTATION--PRESS RELEASE; STAFF REPORT; AND STATEMENT BY THE EXECUTIVE DIRECTOR FOR THE REPUBLIC OF ESTONIA

Under Article IV of the IMF's Articles of Agreement, the IMF holds bilateral discussions with members, usually every year. In the context of the [Consultation Year] Article IV consultation with the Republic of Estonia, the following documents have been released and are included in this package:

January 2017



IMF Country Report No. 17/11

PEOPLE'S REPUBLIC OF CHINA HONG KONG SPECIAL ADMINISTRATIVE REGION

2016 ARTICLE IV CONSULTATION--PRESS RELEASE AND STAFF REPORT

Under Article IV of the IMF's Articles of Agreement, the IMF holds bilateral discussions with members, usually every year. In the context of the 2016 Article IV consultation with the People's Republic of ChinaHong Kong Special Administrative Region, the following documents have been released and are included in this package:

Janua

Remove the /tmp folder once copying process is done

In [9]:
shutil.rmtree(output)

One we have the /txt folder, we can pass the txt to our nlp training model. I will start with tokenizing the text.

In [14]:
# TOKENIZE WORDS FROM TXT FILE

from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import re

countryList = [txt for txt in os.listdir(newOutput) if txt.endswith('.txt')]

selectedCountry = countryList[0]
country = selectedCountry.split('--')[0]
year    = selectedCountry.split('--')[1:]

country="ANGOLA"
txtFp = os.getcwd() + '/txt/'+ selectedCountry
txtF = open(txtFp,errors="ignore")

keyIssues = {}
with open(txtFp,errors="ignore") as fp:
    for line in fp:
        if line.startswith("KEY ISSUES"):
            # 1st readLine start right under KEY ISSUES header
            # each fp.readLine will return a paragraph ended with
            line=fp.readline()
            while 1:
                print(line)
                #print("==>" + line)
                if re.search(country+"$",line):
                    break
                issue = line.replace('\n','').split('.')
                issue = {'{}'.format(issue[0]) : '.'.join(issue[1:])}
                
                keyIssues.update(issue)
                line=fp.readline()


'''                
#print(sections["keyissues"])
# continue to expand this regex for common stop signs in document
docStopSigns = ['--', '\'s', '...']

# our stop list = English stopwords + common punctuation + docStopSigns
stop = stopwords.words('english') 

# read txt file to string and loop word-to-word by word_tokenize
# read bagOfWords to list MINUS ones from the stop list
# finally, join list of string together making a sequences of string
text = ' '.join( w for w in word_tokenize(sections["keyissues"]) if w not in stop)
'''
keyIssues

Context and outlook: The oil price shock that started in mid-2014 has substantially reduced fiscal revenue and exports, with growth coming to a halt and inflation accelerating sharply. This has brought to the forefront the need to address more forcefully vulnerabilities and dependence on oil, and to diversify the economy. The authorities have taken steps to mitigate the impact of the external shock: an 18 percent of GDP improvement in the non-oil primary fiscal balance over 2015-16, mainly through spending cuts including the removal of fuel subsidies, has been implemented; and the kwanza has been devalued against the U.S. dollar by over 40 percent since September 2014, with international reserves being used to smooth the depreciation. However, the exchange rate has been re-pegged since April 2016 leading to an appreciation of the kwanza in real terms, and further policy actions are needed to continue adjusting the economy to the `new normal' in the oil market and to return growth to a 

{'': '',
 ' Adopt a reserve money target consistent with the inflation objective': '  Increase financial sector resilience by strengthening bank supervision, resolution and',
 ' Adopt an improved medium-term fiscal framework (MTFF), focusing on spending rules and a well-designed fiscal stabilization fund to reduce spending pro-cyclicality': '',
 ' Move to a flexible but managed exchange rate regime to urgently address significant imbalances in the foreign exchange market, and develop a clear timetable for removal of exchange restrictions and multiple currency practices': '',
 ' Over the medium term, target an improvement in the non-oil primary fiscal balance by 1 percent of GDP annually consistent with placing debt on a clearly declining path': ' This could be achieved by mobilizing additional non-oil taxes, and reducing current spending and increasing public investment while improving its efficiency.',
 ' Target an overall fiscal deficit for 2017 of less than 2¼ percent of GDP, consis

In [65]:
# FIND 50 MOST COMMON WORDS

# find 50 most frequent words in the txt by FreqDist from nltk
tokens = word_tokenize(text)
nltkText = nltk.Text(tokens)


fd = FreqDist(nltkText)
common50 = fd.most_common(50)
sorted(common50)
print(' '.join(w for w in tokens))

Context outlook : The oil price shock started mid-2014 substantially reduced fiscal revenue exports , growth coming halt inflation accelerating sharply . This brought forefront need address forcefully vulnerabilities dependence oil , diversify economy . The authorities taken steps mitigate impact external shock : 18 percent GDP improvement non-oil primary fiscal balance 2015-16 , mainly spending cuts including removal fuel subsidies , implemented ; kwanza devalued U.S. dollar 40 percent since September 2014 , international reserves used smooth depreciation . However , exchange rate re-pegged since April 2016 leading appreciation kwanza real terms , policy actions needed continue adjusting economy `new normal ' oil market return growth level consistent poverty reduction . Focus consultation : Discussions focused policies tackle macroeconomic imbalances promoting financial stability , economic diversification , growth . Key policy recommendations : Target overall fiscal deficit 2017 less

Since technical terms are usually complicated and containing multiple characters. I attempt to get rid of short words 

In [56]:
# check if word > 5 characters
# lowercase to prevent repeat count: exp: ANGOLA vs angola
longCharacter = [w.lower() for w in nltkText if len(w) > 5]

# Count words freq from new word list and choose most common 50
technicalCommon50 = sorted(FreqDist(longCharacter).most_common(50))
technicalCommon50

[('2015-16', 1),
 ('accelerating', 1),
 ('address', 2),
 ('authorities', 1),
 ('balance', 3),
 ('brought', 1),
 ('coming', 1),
 ('consistent', 4),
 ('context', 1),
 ('dependence', 1),
 ('diversification', 2),
 ('diversify', 1),
 ('economic', 2),
 ('economy', 2),
 ('exchange', 4),
 ('exports', 1),
 ('external', 1),
 ('financial', 2),
 ('fiscal', 7),
 ('forcefully', 1),
 ('forefront', 1),
 ('growth', 3),
 ('imbalances', 2),
 ('impact', 1),
 ('improvement', 3),
 ('improving', 2),
 ('including', 1),
 ('inflation', 2),
 ('kwanza', 2),
 ('mainly', 1),
 ('market', 2),
 ('mid-2014', 1),
 ('mitigate', 1),
 ('non-oil', 4),
 ('outlook', 1),
 ('percent', 5),
 ('policy', 2),
 ('primary', 3),
 ('reduced', 1),
 ('removal', 2),
 ('revenue', 1),
 ('sector', 2),
 ('sharply', 1),
 ('spending', 4),
 ('started', 1),
 ('strengthening', 2),
 ('subsidies', 1),
 ('substantially', 1),
 ('target', 3),
 ('vulnerabilities', 1)]

In [60]:
nltkText.similar('down')

No matches
